# Object Relational Mappers
- maps objects into a relational database
- somewhat complex but very useful
    - not easy to do yourself
- best known one for Python is [SQLAlchemy](http://www.sqlalchemy.org)
    - [ORM tutorial](https://docs.sqlalchemy.org/en/13/orm/tutorial.html)
    - somewhat similar to [Hibernate](https://hibernate.org) for Java
- flask has support for SQLAlchemy


In [1]:
from sqlalchemy import create_engine

# set echo=True for debug info
engine = create_engine('sqlite:///:memory:', echo=False)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

session = Session()

In [2]:
class Student(Base):
    __tablename__ = 'student'
    id = Column(Integer, primary_key=True)
    first = Column(String)
    last = Column(String)
    uni = Column(String)

    def __repr__(self):
        s = f'<Student(first={self.first}, last={self.last}, uni={self.uni})>'
        return s


In [3]:
class Grade(Base):
    __tablename__ = 'grade'
    id = Column(Integer, primary_key=True)
    uni = Column(String)
    course = Column(String)
    grade = Column(Float)

    def __repr__(self):
        s = f'<Grade(uni={self.uni}, course={self.course}, grade={self.grade})>'
        return s


In [4]:
Base.metadata.create_all(engine)
session.commit()

In [5]:
# spath set in database notebook

%store -r
spath

'/var/folders/ct/ftxzjrdd1r5g__vddj6q335h0000gn/T/tmp8bor2b1r'

In [6]:
# make a db connection and get a cursor

import sqlite3 

# must be the same path as in databases notebook

con = sqlite3.connect(spath)
cur = con.cursor()


In [7]:
res = None

with sqlite3.connect(path) as con:
    cur = con.cursor()
    res = list(cur.execute("select first,last,uni from student"))
    
for f,l,u in res:
    session.add(Student(first=f, last=l, uni=u))

In [8]:
res = None

with sqlite3.connect(spath) as con:
    cur = con.cursor()
    res = list(cur.execute("select uni, course, grade from grade"))
    
for u, c, g in res:
    session.add(Grade(uni=u, course=c, grade=g))

In [9]:
session.new

IdentitySet([<Student(first=Maye, last=Quent, uni=mq6173)>, <Student(first=Jana, last=Genny, uni=jg6501)>, <Student(first=Beck, last=Audre, uni=ba3658)>, <Student(first=Andi, last=Katya, uni=ak5529)>, <Student(first=Roth, last=Kirby, uni=rk1433)>, <Student(first=Rory, last=Glory, uni=rg1828)>, <Student(first=Rene, last=Cissy, uni=rc3042)>, <Student(first=Etti, last=Codie, uni=ec4937)>, <Student(first=Dino, last=Suzzy, uni=ds6773)>, <Student(first=Lela, last=Peter, uni=lp9668)>, <Grade(uni=mq6173, course=COMSW 7938, grade=86.04829391231357)>, <Grade(uni=jg6501, course=COMSW 7938, grade=74.01024513974369)>, <Grade(uni=ba3658, course=COMSW 3390, grade=88.15901362780362)>, <Grade(uni=ak5529, course=COMSW 3496, grade=76.75044751220422)>, <Grade(uni=rk1433, course=COMSW 3598, grade=74.16036257877713)>, <Grade(uni=rg1828, course=COMSW 7938, grade=73.17984986490653)>, <Grade(uni=rc3042, course=COMSW 7938, grade=72.05289482403647)>, <Grade(uni=ec4937, course=COMSW 7938, grade=97.44794034395666)

In [10]:
session.commit()

In [11]:
session.new

IdentitySet([])

In [12]:
fstudent = session.query(Student).first()
fstudent

<Student(first=Maye, last=Quent, uni=mq6173)>

In [13]:
# 'dirty' objects, ones that have been modifed
# are persisted 

fstudent.uni = 'zzzz'
session.commit()

In [14]:
session.query(Student).first()

<Student(first=Maye, last=Quent, uni=zzzz)>

In [15]:
# get grades better than 90

session.query(Grade).filter(Grade.grade > 90).all()

[<Grade(uni=ec4937, course=COMSW 7938, grade=97.44794034395666)>,
 <Grade(uni=lp9668, course=COMSW 3390, grade=96.53538398782479)>]

In [16]:
# join grade>90 rows with Student table

session.query(Grade, Student).filter(Grade.grade > 90).filter(Grade.uni == Student.uni).all()


[(<Grade(uni=ec4937, course=COMSW 7938, grade=97.44794034395666)>,
  <Student(first=Etti, last=Codie, uni=ec4937)>),
 (<Grade(uni=lp9668, course=COMSW 3390, grade=96.53538398782479)>,
  <Student(first=Lela, last=Peter, uni=lp9668)>)]